In [1]:
# Import pandas
import pandas as pd
# Import the required dependencies from sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

# Set the column width to view the text message data.
pd.set_option('max_colwidth', 200)

# Import Gradio
import gradio as gr


In [2]:
# Preprocess to handle NaN values
def preprocess_text(text):
    return '' if pd.isna(text) else str(text)

In [3]:
def sms_classification(sms_text_df):
    """
    Perform SMS classification using a pipeline with TF-IDF vectorization and Linear Support Vector Classification.

    Parameters:
    - sms_text_df (pd.DataFrame): DataFrame containing 'text_message' and 'label' columns for SMS classification.

    Returns:
    - text_clf (Pipeline): Fitted pipeline model for SMS classification.

    This function takes a DataFrame with 'text_message' and 'label' columns, splits the data into
    training and testing sets, builds a pipeline with TF-IDF vectorization and Linear Support Vector
    Classification, and fits the model to the training data. 
    The fitted pipeline is returned to make future predictions.
    """

    # Preprocess the 'message' column to handle NaN values
    sms_text_df['text message'] = sms_text_df['text message'].apply(preprocess_text)

    # Set the features variable to the text message column.
    features = sms_text_df['text message']
    
    # Set the target variable to the "label" column.
    target = sms_text_df['label']

    # Split data into training and testing and set the test_size = 33%
    X_train, X_test, y_train, y_test = train_test_split(features, target, 
                                                        test_size=0.33, random_state=50)


    # Build a pipeline to transform the test set to compare to the training set.
    text_clf = Pipeline([
        ('tfidf', TfidfVectorizer(strip_accents='unicode', 
                                  stop_words='english',
                                  lowercase=True)),
        ('clf', LinearSVC())
    ])

    # Fit the model to the transformed training data and return model.
    text_clf.fit(X_train, y_train)
    return text_clf, X_test, y_test

## The X_test and y_test are not used in the current implementation.
## I will modify the sms_classification function to include model evaluation using X_test and y_test.


In [4]:
# Load the dataset into a DataFrame
sms_text_df = pd.read_csv('SMSSpamCollection.csv', sep=',', 
                          names=['label', 'text message'], skiprows=1)

print(sms_text_df.columns)
sms_text_df.head()

Index(['label', 'text message'], dtype='object')


,label,text message
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [5]:
# Call the sms_classification function with the DataFrame and set the result to the "text_clf" variable
#text_clf = sms_classification(sms_text_df)
text_clf, X_test, y_test = sms_classification(sms_text_df)

## error when calling the function, found it could be that the "message" column has NaN values, 
## so I will handle the NaN value before we pass the data to TfidfVectorizer by using a custom preprocessor 
## by converting the NaN values to an empty string, modifying the 'sms_classification' function.

In [6]:
from sklearn.metrics import classification_report, accuracy_score
 # Evaluate the model on the test set
y_pred = text_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Model Accuracy: {accuracy}")
print("Classification Report:")
print(report)


Model Accuracy: 0.9864056552474171
Classification Report:
              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1614
        spam       0.97      0.92      0.94       225

    accuracy                           0.99      1839
   macro avg       0.98      0.96      0.97      1839
weighted avg       0.99      0.99      0.99      1839



In [7]:
# Create a function called `sms_prediction` that takes in the SMS text and predicts the whether the text is "not spam" or "spam". 
# The function should return the SMS message, and say whether the text is "not spam" or "spam".
def sms_prediction(text):
    """
    Predict the spam/ham classification of a given text message using a pre-trained model.

    Parameters:
    - text (str): The text message to be classified.

    Returns:
    - str: A message indicating whether the text message is classified as spam or not.

    This function takes a text message and a pre-trained pipeline model, then predicts the
    spam/ham classification of the text. The result is a message stating whether the text is
    classified as spam or not.
    """
    # Preprocess the input text
    processed_text = preprocess_text(text)

    # Create a variable that will hold the prediction of a new text.
    prediction = text_clf.predict([processed_text])[0]
    
    # Using a conditional if the prediction is "ham" return the message:
    # f'The text message: "{text}", is not spam.' Else, return f'The text message: "{text}", is spam.'
    if prediction == 'ham':
        return f'The text message: "{text}", is not spam.'
    else:
        return f'The text message: "{text}", is spam.'

In [8]:
# Create a sms_app that takes a textbox for the inputs and has a textbox for the output.  
# Povide labels for each textbox. 
sms_app = gr.Interface(
    fn=sms_prediction,
    inputs=gr.Textbox(label="Enter SMS text"),
    outputs=gr.Textbox(label="Prediction"),
    title="SMS Spam Classifier",
    description="Enter a text message to classify it as spam or not spam."
)
    
# Launch the app.
sms_app.launch(share=True)

Running on local URL:  http://127.0.0.1:7860


/opt/anaconda3/envs/dev/lib/python3.10/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.1, however version 5.0.1 is available, please upgrade. 
--------
  warnings.warn(


Running on public URL: https://ac796ef35ce788e0b5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## Test the following text messages. 

---

1. You are a lucky winner of $5000!
2. You won 2 free tickets to the Super Bowl.
3. You won 2 free tickets to the Super Bowl text us to claim your prize.
4. Thanks for registering. Text 4343 to receive free updates on medicare.